In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.patches import Circle


import timeit
import numba
from numba import jit, njit

from iteration_methods import grad_shaf_solver, gs_comp_solver 
from basic import*

In [2]:
#Set output options
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
@njit
def source(r, th, u, params = (1, 0, 0.5)):

    sigma, s, uc = params

    f1 = -sigma*s**2*(u-uc)**(2*sigma-1)
    
    return f1

@njit
def u_analytical(r, th):

    uan = np.sin(th)**2/r 

    return uan

In [4]:
#Setup the parameters for the solver. Function grid() is in the basic.py module


       
#extra parameters that may be needed
sigma = 1.1
s = 0.
uc = 0.5
params = (sigma , s, uc)
# params1 = (sigma , 0., uc)
# params2 = (1. , s, uc)

#max number of iteration
iterations = 10000                              

#desired tolerance
tolerance = 1.e-8                                   

In [5]:
#jit the solver function
gs_comp_solver = jit(nopython=True)(gs_comp_solver)
grad_shaf_solver = jit(nopython=True)(grad_shaf_solver)

In [6]:
#Calculate, time and qualify the solution in the compact coordinates

r_s = 1.
error_to_an = []
Nmax=200
N = np.array([25, 50, 100, 200, 400])
# N = [100]*5
u_list=[]
resolution = []
omega = np.asarray(np.arange(1.8, 2., 0.025))
k_list=[]
# for n in range(len(N)):
# # for i in range(len(omega)):
#     print('Nr={:}'.format(N[n]))
    
#The grid of the problem
Q, TH, q, th, dq, dth = polar_grid(th_I=0, th_F=np.pi, r_I=0, r_F=r_s, Nr=50, Nth=50)
R, r = r_s**2/Q, r_s**2/q

resolution.append(dq)

u_an = u_analytical(r, th)
u_guess = 2*np.sin(th)**2/r       
boundary = ((0, np.sin(TH)**2/r_s), (0., 0.))                                   
omega_opt = 1.975     

start = timeit.default_timer()

u, k, rel_diff, conv_hist = gs_comp_solver(np.eye(2), source, (q, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

elapsed = timeit.default_timer() - start

u_list.append(u)
k_list.append(k)
#Relative error to analytical
error = la.norm(u - u_an, 2)/la.norm(u_an, 2)
error_to_an.append(error)

#print solution
print('SOR solver compact \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
    '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error, elapsed))
print('-'*50)

50 0.02
SOR solver compact 
Number of iterations: 735 
Last Relative difference: 9.650E-09
Error to analytical: 6.018E-04s 
Elapsed time: 4.69E+00s 
--------------------------------------------------


In [7]:
#Calculate, time and qualify the solution in the normal coordinates

r_s = 1
error_to_an1 = []
Nmax=400
N = np.array([25, 50, 100, 200, 400])
# N = [50]
u1_list=[]
resolution1 = []
# for n in range(len(N)):
#     print('Nr={:}'.format(N[n]))
    
#The grid of the problem
R1, TH1, r1, th1, dr1, dth1 = polar_grid(th_I=0, th_F=np.pi, r_I=r_s, r_F=10*r_s, Nr=50, Nth=50)            


resolution1.append(dr1)

u_an1 = u_analytical(r1, th1)
u_guess1 = 2*np.sin(th1)**2/r1       
boundary1 = ((np.sin(TH1)**2/r_s, 0), (0., 0.))                                   
omega_opt1 = 1.975     

start = timeit.default_timer()

u1, k1, rel_diff1, conv_hist1 = grad_shaf_solver(np.eye(2), source, (r1, th1), init_guess=u_guess1, boundary=boundary1,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt1,                                                                           params=params)

elapsed = timeit.default_timer() - start

u1_list.append(u1)

#Relative error to analytical
error1 = la.norm(u1 - u_an1, 2)/la.norm(u_an1, 2)
error_to_an1.append(error1)

#print solution
print('SOR solver normal \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
    '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k1, rel_diff1, error1, elapsed))
print('-'*50)

SOR solver normal 
Number of iterations: 677 
Last Relative difference: 9.566E-09
Error to analytical: 5.280E-03s 
Elapsed time: 3.71E+00s 
--------------------------------------------------


In [8]:
# fig3, ax3 = plt.subplots()

# ax3.plot(N, error_to_an, label='compact', c='b')
# ax3.plot(N, error_to_an1, label='normal', c='r')
# ax3.scatter(N, error_to_an, c='b')
# ax3.scatter(N, error_to_an1, c='r')

# ax3.plot(N, 1/N**2, label='$N^{-2}$', c='g')
# ax3.scatter(N, 1/N**2, c='g')

# ax3.set_yscale('log')
# ax3.set_xscale('log')

# ax3.set_title('Convergence')
# ax3.set_xlabel('Number of radial grid points Nr')
# ax3.set_ylabel('Relative error to analytical')
# ax3.legend()

ValueError: x and y must have same first dimension, but have shapes (5,) and (1,)

In [12]:
fig2, ax2 = plt.subplots()

levels=20
colormap=cm.viridis

cont = ax2.contour(r*np.sin(th), r*np.cos(th), u, levels=levels, cmap=colormap)
cont_q = ax2.contour(q*np.sin(th), q*np.cos(th), u, levels=levels, cmap=colormap, alpha=0.5)

cont1 = ax2.contour(r1*np.sin(th1), r1*np.cos(th1), u1, levels=levels, cmap=colormap, linestyles = 'dashed')

# cont2 = ax2.contour(r*np.sin(th), r*np.cos(th), u2, levels=levels, cmap=colormap, linestyles = 'dotted') 

# cont_c = ax2.contour(r_s**2/q*np.sin(th), r_s**2/q*np.cos(th), u, levels=[uc], colors='red')
# cont_c1 = ax2.contour(r*np.sin(th), r*np.cos(th), u1, levels=[uc], colors='green')
# cont_c2 = ax2.contour(r*np.sin(th), r*np.cos(th), u2, levels=[uc], colors='yellow')
cont_an = ax2.contour(r*np.sin(th), r*np.cos(th), u_an, levels=levels, colors='black', zorder=-1)
# cont_an = ax2.contour(r1*np.sin(th1), r1*np.cos(th1), u_an1, levels=levels, cmap=colormap, alpha=0.5)

ax2.add_patch(Circle((0.,0.), r_s, color='b', zorder=100, fill=False))

fig_lim = 10
ax2.set_ylim(top=fig_lim*r_s*np.cos(TH[0])/2, bottom=fig_lim*r_s*np.cos(TH[-1])/2)
ax2.set_xlim(left=-0.1, right=fig_lim*r_s)

ax2.set_title('Compact Vs. Normal')
h,_ = cont.legend_elements()
h_q, _ = cont_q.legend_elements()
ax2.legend([h[0], h_q[0]], ['normal', 'compact'], loc='best')
h1,_ = cont1.legend_elements()
# # h2,_ = cont2.legend_elements()
h_an,_ = cont_an.legend_elements()
ax2.legend([h[0], h1[0], h_an[0], h_q[0]], ['compact', 'normal', 'analytical', 'raw_compact'], loc='best')